In [ ]:
from xml.etree import ElementTree as ET

import os
import numpy as np

In [ ]:
def extract_bounding_boxes(xml_file):
    # create element tree object
    tree = ET.parse(xml_file)
  
    # get root element
    root = tree.getroot()
    bbox_xmin = root.findall("./object/bndbox/xmin")[0].text
    bbox_ymin = root.findall("./object/bndbox/ymin")[0].text
    bbox_xmax = root.findall("./object/bndbox/xmax")[0].text
    bbox_ymax = root.findall("./object/bndbox/ymax")[0].text

    return int(bbox_xmin), int(bbox_ymin), int(bbox_xmax), int(bbox_ymax)

In [ ]:
def calc_intersection_area(bbox1_xmin, bbox2_xmin, bbox1_ymin, bbox2_ymin, bbox1_xmax, 
                           bbox2_xmax, bbox1_ymax, bbox2_ymax):
    # determine the (x, y)-coordinates of the intersection rectangle
    inter_xMin = max(bbox1_xmin, bbox2_xmin)
    inter_yMin = max(bbox1_ymin, bbox2_ymin)
    
    inter_xMax = min(bbox1_xmax, bbox2_xmax)
    inter_yMax = min(bbox1_ymax, bbox2_ymax)

    # return the area of intersection rectangle
    return max(0, inter_xMax - inter_xMin + 1) * max(0, inter_yMax - inter_yMin + 1)


In [ ]:
def process_slices(annot_dir1, annot_dir2):
    # Get similar files between two annotators
    files_annot1 = [file for file in os.listdir(annot_dir1) if not file.endswith(".jpg")]
    files_annot2 = [file for file in os.listdir(annot_dir2) if not file.endswith(".jpg")]

    # Get unique files
    files = set(files_annot1).intersection(files_annot2)

    iou_scores_per_file = []
    for file in files:
        xmin_annot1, ymin_annot1, xmax_annot1, ymax_annot1 = extract_bounding_boxes(f"{annot_dir1}/{file}")
        xmin_annot2, ymin_annot2, xmax_annot2, ymax_annot2 = extract_bounding_boxes(f"{annot_dir2}/{file}")
    
        intersection_area = calc_intersection_area(xmin_annot1, xmin_annot2, ymin_annot1, ymin_annot2, xmax_annot1,
                                                  xmax_annot2, ymax_annot1, ymax_annot2)
        
        annot1_area = (xmax_annot1 - xmin_annot1 + 1) * (ymax_annot1 - ymin_annot1 + 1)
        annot2_area = (xmax_annot2 - xmin_annot2 + 1) * (ymax_annot2 - ymin_annot2 + 1)
        
        iou = intersection_area / (annot1_area + annot2_area - intersection_area)
        iou_scores_per_file.append(iou)
    return np.mean(iou_scores_per_file)

In [ ]:
annot1_dir = "/Users/nicklittlefield/OneDrive - University of Pittsburgh/Dataset_Knee_JointArea_Localization_MRIs/Batch#2/#AnnotatorNumberOne_Ismaeel"
annot2_dir = "/Users/nicklittlefield/OneDrive - University of Pittsburgh/Dataset_Knee_JointArea_Localization_MRIs/Batch#2/#AnnotatorNumberTwo_Sanyukta"

In [ ]:
annot1_mri_dirs = os.listdir(annot1_dir)
annot2_mri_dirs = os.listdir(annot2_dir)
final_mri_dirs = set([mri for mri in annot1_mri_dirs if mri != ".DS_Store"]).intersection(annot2_mri_dirs)

In [ ]:
ious = []

for sag_mri in final_mri_dirs:
    annot1 = f"{annot1_dir}/{sag_mri}"
    annot2 = f"{annot2_dir}/{sag_mri}"
    
    overall_iou = process_slices(annot1, annot2)
    ious.append(overall_iou)

In [ ]:
np.mean(ious)